### F1

http://ergast.com/mrd/

In [1]:
import pyodbc
from sqlalchemy import create_engine
import sys
import pandas as pd
import requests
import time
from pprint import pprint

In [2]:
#Necesitamos encontrar el ultimo raceId de la tabla lap_Times
try:
    connect = pyodbc.connect("""
        DRIVER={SQL Server};
        SERVER=LAPTOP-5EGKJ2TF;
        DATABASE=Formula1;
        Trusted_Connection=yes')
        """)
    sql_query = 'SELECT MAX(raceId) FROM lapTimes'
    
    #Dataset with headers
    max_raceId_df = pd.read_sql(sql_query, connect)
    
    # Obtener el valor máximo de raceId como una variable
    max_raceId = max_raceId_df.iloc[0, 0]

    cursor = connect.cursor()
    cursor.execute(sql_query)
    connect.commit()

except Exception as ex:
    print(ex)
finally :
    cursor.close()
    connect.close()
    print('Conexión finalizada')

Conexión finalizada


C:\Users\maria\AppData\Local\Temp\ipykernel_36260\1675199318.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  max_raceId_df = pd.read_sql(sql_query, connect)


In [3]:
max_raceId

1110

In [4]:
#Necesitamos encontrar el año y la carrera del siguiente valor para ver si ya fue cargado
try:
    connect = pyodbc.connect("""
        DRIVER={SQL Server};
        SERVER=LAPTOP-5EGKJ2TF;
        DATABASE=Formula1;
        Trusted_Connection=yes')
        """)
    sql_query = '''
                    SELECT 
                        raceId,
                        year,
                        round
                    FROM races
                    WHERE raceID = (SELECT MAX(raceId)+1 FROM lapTimes)
                '''
    
    #Dataset with headers
    nextMax_raceId_df = pd.read_sql(sql_query, connect)
    

    cursor = connect.cursor()
    cursor.execute(sql_query)
    connect.commit()

except Exception as ex:
    print(ex)
finally :
    cursor.close()
    connect.close()
    print('Conexión finalizada')

Conexión finalizada


C:\Users\maria\AppData\Local\Temp\ipykernel_36260\1225873436.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nextMax_raceId_df = pd.read_sql(sql_query, connect)


In [5]:
año = nextMax_raceId_df.iloc[0, 1]
carrera = nextMax_raceId_df.iloc[0, 2]
print (f'Necesitamos buscar la informacion en la API del raceId {max_raceId+1}; año : {año} y carrera {carrera}')

Necesitamos buscar la informacion en la API del raceId 1111; año : 2023 y carrera 13


In [2]:
#Cambiarlo luegooooooooo
max_raceId = 1110
año = 2023
carrera = 12

###     Para mas adelante podriamos en hacer un while adicional que me permita sacar todos los valores de vueltas de todas las carreras del ultimo año.
####    Por ahora lo haremos solo por la carrera encontrada

In [3]:
try:
    i = 1 #Empieza siempre en la vuelta 1
    laps_ds = [] #Creamos el dataset vacio que almacenar toda la informacion
    while i < 100 : # Ninguna carrera dura mas de 100 vueltas. El while parara cuando se encuentre un error
        url = f'http://ergast.com/api/f1/{año}/{carrera}/laps/{i}.json'
        res=requests.get(url)
        lapsdata = res.json()['MRData']['RaceTable']['Races'][0]['Laps']
        
        for lap in lapsdata:
            Nlap = lap['number']
            timings = lap['Timings']
            for timing in timings:
                driver_id = timing['driverId']
                position = timing['position']
                time = timing['time']
                #print(f"Driver: {driver_id}, Position: {position},Lap: {Nlap}, Time: {time}")
                laps_ds.append({
                    'raceId': max_raceId,
                    'driver': driver_id, 
                    'position': position, 
                    'lap': Nlap, 
                    'time': time
                })
        i += 1     

except Exception as ex:
    print(ex)
finally :
    print('No hay mas datos por leer')

list index out of range
No hay mas datos por leer


In [4]:
laps_total=pd.DataFrame(laps_ds)
laps_total

,raceId,driver,position,lap,time
0,1110,perez,1,1,1:53.295
1,1110,leclerc,2,1,1:54.584
2,1110,hamilton,3,1,1:55.210
3,1110,max_verstappen,4,1,1:55.659
4,1110,sainz,5,1,1:57.901
...,...,...,...,...,...
810,1110,albon,14,44,1:51.451
811,1110,kevin_magnussen,15,44,1:53.438
812,1110,ricciardo,16,44,1:52.265
813,1110,sargeant,17,44,1:52.590


In [5]:
if laps_total.empty:
        print('No hay valores a actualizar')
        sys.exit()        

In [6]:
# Todas las columnas estan como object
laps_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 815 entries, 0 to 814
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   raceId    815 non-null    int64 
 1   driver    815 non-null    object
 2   position  815 non-null    object
 3   lap       815 non-null    object
 4   time      815 non-null    object
dtypes: int64(1), object(4)
memory usage: 32.0+ KB


In [7]:
# Cambiamos el tipo de dato de las columnas position y lap
laps_total['position'] = laps_total['position'].astype(int)
laps_total['lap'] = laps_total['lap'].astype(int)

In [8]:
# Cambiamos el tipo de dato de las columna time al formato time
laps_total['time'] = pd.to_datetime(laps_total['time'], format='%M:%S.%f')

In [9]:
# A partir de la columna time en formato tiempo, generamos una nueva columna que transforma este tiempo en milisegundos
#pd.to_datetime((laps_total['time'].dt.minute*60*1000 + laps_total['time'].dt.second*1000 + laps_total['time'].dt.microsecond/1000), unit='ms')
laps_total['milliseconds'] = (laps_total['time'].dt.minute*60*1000 + laps_total['time'].dt.second*1000 + laps_total['time'].dt.microsecond/1000).astype(int)

In [10]:
laps_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 815 entries, 0 to 814
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   raceId        815 non-null    int64         
 1   driver        815 non-null    object        
 2   position      815 non-null    int32         
 3   lap           815 non-null    int32         
 4   time          815 non-null    datetime64[ns]
 5   milliseconds  815 non-null    int32         
dtypes: datetime64[ns](1), int32(3), int64(1), object(1)
memory usage: 28.8+ KB


In [15]:
#Necesitamos encontrar el driverId en vez de la columna driver. Necesitamos importar la data de drivers de SQL Server
try:
    connect = pyodbc.connect("""
        DRIVER={SQL Server};
        SERVER=LAPTOP-5EGKJ2TF;
        DATABASE=Formula1;
        Trusted_Connection=yes')
        """)
    sql_query = 'SELECT * FROM drivers'
    
    #Dataset with headers
    drivers = pd.read_sql(sql_query, connect)

    cursor = connect.cursor()
    cursor.execute(sql_query)
    connect.commit()

except Exception as ex:
    print(ex)
finally :
    cursor.close()
    connect.close()
    print('Conexión finalizada')

Conexión finalizada


C:\Users\maria\AppData\Local\Temp\ipykernel_36260\368386801.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  drivers = pd.read_sql(sql_query, connect)


In [16]:
drivers.tail(3)

,driverId,driverRef,number,code,firstName,lastName,birth,nationality,url
854,856,de_vries,21.0,DEV,Nyck,de Vries,1995-02-06,Dutch,http://en.wikipedia.org/wiki/Nyck_de_Vries
855,857,piastri,81.0,PIA,Oscar,Piastri,2001-04-06,Australian,http://en.wikipedia.org/wiki/Oscar_Piastri
856,858,sargeant,2.0,SAR,Logan,Sargeant,2000-12-31,American,http://en.wikipedia.org/wiki/Logan_Sargeant


In [17]:
drivers.groupby('driverRef')['driverId'].count().sort_values() #No hay DriverRef repetidos :)

driverRef
Cannoc       1
naspetti     1
nasr         1
natili       1
naylor       1
            ..
gasly        1
gavin        1
gaze         1
gendebien    1
zunino       1
Name: driverId, Length: 857, dtype: int64

In [18]:
laps_total = laps_total.merge(drivers[['driverRef', 'driverId']], left_on='driver', right_on='driverRef', how='left')

In [19]:
laps_total = laps_total[['raceId', 'driverId', 'lap', 'position', 'time', 'milliseconds']]
laps_total.head(3)

,raceId,driverId,lap,position,time,milliseconds
0,1110,815,1,1,1900-01-01 00:01:53.295,113295
1,1110,844,1,2,1900-01-01 00:01:54.584,114584
2,1110,1,1,3,1900-01-01 00:01:55.210,115210


In [20]:
laps_total.columns = ['raceId', 'driverId', 'lap', 'racePosition', 'lapTime', 'milliseconds']

In [21]:
laps_total.head(3)

,raceId,driverId,lap,racePosition,lapTime,milliseconds
0,1110,815,1,1,1900-01-01 00:01:53.295,113295
1,1110,844,1,2,1900-01-01 00:01:54.584,114584
2,1110,1,1,3,1900-01-01 00:01:55.210,115210


In [22]:
#Ahora intentaremos cargar la informacion en la tabla staging laps_total en SQL Server
try:
    server='LAPTOP-5EGKJ2TF'
    database='Formula1'
    username='LAPTOP-5EGKJ2TF\maria'
    
    alchemyEngine = create_engine(f'mssql+pyodbc://{username}@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')            
    
    laps_total.to_sql('laps_total', alchemyEngine, if_exists='append', index=False, chunksize=1000)
    
except Exception as ex:
    print(ex)
finally :
    print('Conexión finalizada')

Conexión finalizada


In [23]:
# Usar tabla staging laps_total, para actualizar nuestra tabla en produccion lapTimes en SQL Server
try:
    connect = pyodbc.connect("""
                                DRIVER={SQL Server};
                                SERVER=LAPTOP-5EGKJ2TF;
                                DATABASE=Formula1;
                                Trusted_Connection=yes')
                            """)

    sql_query = '''
                    MERGE lapTimes AS target
                    USING laps_total AS source
                    ON target.raceId = source.raceId AND target.driverId = source.driverId AND target.lap = source.lap
                    WHEN NOT MATCHED BY target THEN
                    --<accion cuando no coinciden por destino>
                        INSERT (
                            raceId,
                            driverId,
                            lap,
                            racePosition,
                            lapTime,
                            milliseconds
                        )
                        VALUES (
                            source.raceId,
                            source.driverId,
                            source.lap,
                            source.racePosition,
                            source.lapTime,
                            source.milliseconds
                        ); -- END MERGE
                    
                    DROP TABLE laps_total
                '''  
    
    cursor = connect.cursor()
    cursor.execute(sql_query)
    connect.commit()
    
except Exception as ex:
    print(ex)
finally :
    cursor.close()
    connect.close()
    print('Conexión finalizada')

Conexión finalizada
